In [ ]:
import altair as alt
import datetime as dt
import pandas as pd
import requests


from toolz.curried import *

alt.data_transformers.disable_max_rows()

In [ ]:
BASE_URL = "https://thalex.com/api/v2/public"
endpoint = "mark_price_historical_data"
url = f"{BASE_URL}/{endpoint}"

params = lambda instrument_name: {
    "from": int(dt.datetime(2025, 2, 27, 8, 0).timestamp()),
    "to": int(dt.datetime(2025, 3, 3, 0, 0).timestamp()),
    "resolution": "15m",
    "instrument_name": instrument_name,
}

COLUMNS = [
    "ts",
    "mark_price_open",
    "mark_price_high",
    "mark_price_low",
    "mark_price_close",
    "iv_open",
    "iv_high",
    "iv_low",
    "iv_close",
]


instrument_names = [
    "BTC-28MAR25-85000-C",
    "BTC-28MAR25-86000-C",
    "BTC-28MAR25-87000-C",
    "BTC-28MAR25-88000-C",
    "BTC-28MAR25-90000-C",
    "BTC-28MAR25-92000-C",
    "BTC-28MAR25-93000-C",
    "BTC-28MAR25-95000-C",
    "BTC-28MAR25-95000-C",
    "BTC-28MAR25-97000-C",
    "BTC-28MAR25-100000-C",
]

df = (
    pipe(
        {name: requests.get(url, params=params(name)) for name in instrument_names},
        valmap(requests.Response.json),
        valmap(get_in(["result", "mark"])),
        valmap(curry(pd.DataFrame, columns=COLUMNS)),
        pd.concat,
    )
    .droplevel(1)
    .reset_index(names=["instrument_name"])
    .assign(date_time=lambda df: pd.to_datetime(df["ts"], unit="s"))
    .assign(k=lambda df: df["instrument_name"].map(lambda s: int(s.split("-")[2])))
)

df.tail()

In [ ]:
iv_chart = (
    alt.Chart(
        df,
        width=1600,
        height=900,
        title=alt.Title(
            f"{df.iloc[0]['instrument_name'][:11]}",
            fontSize=22,
            offset=20,
            subtitle=f"{df['date_time'].iloc[0].strftime('%d %b %y %I:%M %p')} — {df['date_time'].iloc[-1].strftime('%d %b %y %I:%M %p')}",
            subtitleFontSize=18,
            subtitlePadding=20,
        ),
    )
    .mark_circle(opacity=0.9, stroke="black", strokeWidth=0.3, size=100)
    .encode(
        x=alt.X("date_time:T", title="Date"),
        y=alt.Y("iv_close", scale=alt.Scale(zero=False), title="IV"),
        color=alt.Color(
            "instrument_name",
            scale=alt.Scale(scheme="blueorange"),
            sort=alt.SortField("k"),
        ),
    )
)


iv_chart

In [ ]:
mark_chart = (
    alt.Chart(
        df,
        width=1600,
        height=900,
        title=alt.Title(
            f"{df.iloc[0]['instrument_name'][:11]}",
            fontSize=22,
            offset=20,
            subtitle=f"{df['date_time'].iloc[0].strftime('%d %b %y %I:%M %p')} — {df['date_time'].iloc[-1].strftime('%d %b %y %I:%M %p')}",
            subtitleFontSize=18,
            subtitlePadding=20,
        ),
    )
    .mark_circle(opacity=0.9, stroke="black", strokeWidth=0.3, size=100)
    .encode(
        x=alt.X("date_time:T", title="Date"),
        y=alt.Y("mark_price_close", scale=alt.Scale(zero=False), title="Mark Price"),
        color=alt.Color(
            "instrument_name",
            scale=alt.Scale(scheme="blueorange"),
            sort=alt.SortField("k"),
        ),
    )
)

mark_chart

In [ ]:
(mark_chart.properties(height=600) & iv_chart.properties(height=600))